In [1]:
!pip install sqlalchemy pandas
!pip install matplotlib
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.9 MB/s eta 0:00:00a 0:00:01


In [23]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.orm import declarative_base, sessionmaker
import pandas as pd
from sqlalchemy import text
from sqlalchemy.inspection import inspect
from sqlalchemy.orm import aliased
from sqlalchemy import func
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
engine = create_engine("sqlite:///disaster_data.db", echo=True)

In [25]:
Base = declarative_base()

In [26]:
Session = sessionmaker(bind=engine)
session = Session()

In [27]:
fema_df = pd.read_csv("cleaned_fema_data.csv")
climate_df = pd.read_csv("cleaned_climate_data.csv")
bls_df = pd.read_csv("cleaned_bls_data.csv")

In [28]:
#FEMA Table

class Disaster(Base):
    __tablename__ = "disasters"

    disasterNumber = Column(Integer, primary_key=True)
    femaDeclarationString = Column(String)
    state = Column(String(2))
    declarationType = Column(String(10))
    declarationDate = Column(Date)
    incidentType = Column(String)
    incidentBeginDate = Column(Date)
    incidentEndDate = Column(Date)
    region = Column(Integer)

In [29]:
#Climate Table

class Climate(Base):
    __tablename__ = "climate"

    id = Column(Integer, primary_key=True, autoincrement=True)  # Unique ID
    year = Column(Integer, nullable=False)  # Allow duplicates
    avg_temp = Column(Float)
    co2_emissions = Column(Float)
    sea_level_rise = Column(Float)
    rainfall = Column(Float)
    population = Column(Float)
    renewable_energy = Column(Float)
    extreme_weather_events = Column(Integer)
    forest_area = Column(Float)

In [30]:
#BLS Tables

class BLS(Base):
    __tablename__ = "bls"

    id = Column(Integer, primary_key=True, autoincrement=True)
    series_id = Column(String)
    year = Column(Integer)
    period = Column(String)
    value = Column(Float)

In [31]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)
print("✅ Tables created successfully!")

2025-02-23 10:14:58,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-23 10:14:58,603 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("disasters")
2025-02-23 10:14:58,604 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-23 10:14:58,605 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("disasters")
2025-02-23 10:14:58,605 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-23 10:14:58,606 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("climate")
2025-02-23 10:14:58,606 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-23 10:14:58,607 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("climate")
2025-02-23 10:14:58,607 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-23 10:14:58,607 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bls")
2025-02-23 10:14:58,607 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-23 10:14:58,608 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("bls")
2025-02-23 10:14:58,608 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [32]:
#adjust FEMA data

fema_df["declarationDate"] = pd.to_datetime(fema_df["declarationDate"])
fema_df["incidentBeginDate"] = pd.to_datetime(fema_df["incidentBeginDate"])
fema_df["incidentEndDate"] = pd.to_datetime(fema_df["incidentEndDate"])

fema_df = fema_df.drop_duplicates(subset=["disasterNumber"])

In [33]:
#adjust climate data

climate_df.rename(columns={
    "Year": "year",
    "Avg Temperature (°C)": "avg_temp",
    "CO2 Emissions (Tons/Capita)": "co2_emissions",
    "Sea Level Rise (mm)": "sea_level_rise",
    "Rainfall (mm)": "rainfall",
    "Population": "population",
    "Renewable Energy (%)": "renewable_energy",
    "Extreme Weather Events": "extreme_weather_events",
    "Forest Area (%)": "forest_area"
}, inplace=True)

# Ensure numeric data is in the correct format
numeric_columns = ["year", "avg_temp", "co2_emissions", "sea_level_rise",
                   "rainfall", "population", "renewable_energy",
                   "extreme_weather_events", "forest_area"]

climate_df[numeric_columns] = climate_df[numeric_columns].astype(float)

In [34]:
#adjust bls

bls_df["year"] = bls_df["year"].astype(int)
bls_df["value"] = bls_df["value"].astype(float)

In [35]:
#insert FEMA data

disasters = [
    Disaster(
        disasterNumber=row["disasterNumber"],
        femaDeclarationString=row.get("femaDeclarationString", None),
        state=row.get("state", None),
        declarationType=row.get("declarationType", None),
        declarationDate=row.get("declarationDate", None),
        incidentType=row.get("incidentType", None),
        incidentBeginDate=row.get("incidentBeginDate", None),
        incidentEndDate=row.get("incidentEndDate", None),
        region=row.get("region", None)
    )
    for _, row in fema_df.iterrows()
]

session.add_all(disasters)
session.commit()
print("✅ FEMA data inserted successfully!")

2025-02-23 10:15:03,841 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-23 10:15:03,846 INFO sqlalchemy.engine.Engine INSERT INTO disasters ("disasterNumber", "femaDeclarationString", state, "declarationType", "declarationDate", "incidentType", "incidentBeginDate", "incidentEndDate", region) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-02-23 10:15:03,846 INFO sqlalchemy.engine.Engine [generated in 0.00167s] [(5453, 'FM-5453-CA', 'CA', 'FM', '2022-09-09', 'Fire', '2022-09-09', '2022-09-26', 9), (5451, 'FM-5451-CA', 'CA', 'FM', '2022-09-06', 'Fire', '2022-09-05', '2022-09-14', 9), (5450, 'FM-5450-CA', 'CA', 'FM', '2022-09-02', 'Fire', '2022-09-02', '2022-09-09', 9), (5445, 'FM-5445-CA', 'CA', 'FM', '2022-07-23', 'Fire', '2022-07-22', '2022-08-03', 9), (5439, 'FM-5439-CA', 'CA', 'FM', '2022-05-12', 'Fire', '2022-05-11', '2022-05-16', 9), (5419, 'FM-5419-CA', 'CA', 'FM', '2021-10-13', 'Fire', '2021-10-12', '2021-11-17', 9), (5417, 'FM-5417-CA', 'CA', 'FM', '2021-09-23', 'Fire', '2021-09-

In [36]:
#insert climate data

climate_records = [
    Climate(
        year=int(row["year"]),
        avg_temp=row["avg_temp"],
        co2_emissions=row["co2_emissions"],
        sea_level_rise=row["sea_level_rise"],
        rainfall=row["rainfall"],
        population=row["population"],
        renewable_energy=row["renewable_energy"],
        extreme_weather_events=int(row["extreme_weather_events"]),
        forest_area=row["forest_area"]
    )
    for _, row in climate_df.iterrows()
]

session.add_all(climate_records)
session.commit()
print("✅ Climate data inserted successfully!")

2025-02-23 10:15:05,257 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-23 10:15:05,259 INFO sqlalchemy.engine.Engine INSERT INTO climate (year, avg_temp, co2_emissions, sea_level_rise, rainfall, population, renewable_energy, extreme_weather_events, forest_area) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING id
2025-02-23 10:15:05,259 INFO sqlalchemy.engine.Engine [generated in 0.00029s (insertmanyvalues) 1/73 (ordered; batch not supported)] (2019, 31.0, 4.8, 4.2, 2407.0, 107364344.0, 49.2, 8, 31.0)
2025-02-23 10:15:05,260 INFO sqlalchemy.engine.Engine INSERT INTO climate (year, avg_temp, co2_emissions, sea_level_rise, rainfall, population, renewable_energy, extreme_weather_events, forest_area) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING id
2025-02-23 10:15:05,260 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/73 (ordered; batch not supported)] (2003, 27.4, 1.8, 2.2, 766.0, 798905927.0, 27.4, 10, 14.6)
2025-02-23 10:15:05,261 INFO sqlalchemy.engine.Engine INSERT INTO climate (

In [37]:
#insert BLS data

bls_records = [
    BLS(
        series_id=row["series_id"],
        year=int(row["year"]),
        period=row["period"],
        value=row["value"]
    )
    for _, row in bls_df.iterrows()
]

session.add_all(bls_records)
session.commit()
print("✅ BLS data inserted successfully!")

2025-02-23 10:15:06,263 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-23 10:15:06,267 INFO sqlalchemy.engine.Engine INSERT INTO bls (series_id, year, period, value) VALUES (?, ?, ?, ?) RETURNING id
2025-02-23 10:15:06,268 INFO sqlalchemy.engine.Engine [generated in 0.00055s (insertmanyvalues) 1/524 (ordered; batch not supported)] ('LAUMT063108000000003', 2024, 'M11', 5.4)
2025-02-23 10:15:06,268 INFO sqlalchemy.engine.Engine INSERT INTO bls (series_id, year, period, value) VALUES (?, ?, ?, ?) RETURNING id
2025-02-23 10:15:06,269 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/524 (ordered; batch not supported)] ('LAUMT063108000000003', 2024, 'M10', 5.6)
2025-02-23 10:15:06,269 INFO sqlalchemy.engine.Engine INSERT INTO bls (series_id, year, period, value) VALUES (?, ?, ?, ?) RETURNING id
2025-02-23 10:15:06,269 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/524 (ordered; batch not supported)] ('LAUMT063108000000003', 2024, 'M09', 5.5)
2025-02-23 10:15:06,269 INFO sqlalchem

In [38]:
#confirming rows in tables

for table in ["disasters", "climate", "bls"]:
    count = session.execute(text(f"SELECT COUNT(*) FROM {table}")).scalar()
    print(f"📊 {table} table has {count} rows")

2025-02-23 10:15:08,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-23 10:15:08,561 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM disasters
2025-02-23 10:15:08,561 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ()
📊 disasters table has 327 rows
2025-02-23 10:15:08,563 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM climate
2025-02-23 10:15:08,563 INFO sqlalchemy.engine.Engine [generated in 0.00035s] ()
📊 climate table has 73 rows
2025-02-23 10:15:08,564 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM bls
2025-02-23 10:15:08,564 INFO sqlalchemy.engine.Engine [generated in 0.00030s] ()
📊 bls table has 524 rows


In [39]:
#creating table Aliases

climate_alias = aliased(Climate)
bls_alias = aliased(BLS)

In [40]:
#joining climate disasters with BLS by year

query = session.query(
    climate_alias.year.label("year"),
    func.avg(climate_alias.avg_temp).label("avg_temp"),
    func.avg(climate_alias.co2_emissions).label("co2_emissions"),
    func.avg(climate_alias.sea_level_rise).label("sea_level_rise"),
    func.avg(climate_alias.rainfall).label("avg_rainfall"),
    func.avg(climate_alias.renewable_energy).label("renewable_energy"),
    func.avg(climate_alias.extreme_weather_events).label("extreme_weather_events"),
    func.avg(bls_alias.value).label("avg_unemployment_rate")  # Aggregate unemployment rate
).join(
    bls_alias, climate_alias.year == bls_alias.year
).group_by(
    climate_alias.year
).order_by(
    climate_alias.year
)

In [41]:
climate_bls_df = pd.DataFrame(query.all(), columns=[
    "year", "avg_temp", "co2_emissions", "sea_level_rise",
    "avg_rainfall", "renewable_energy", "extreme_weather_events",
    "avg_unemployment_rate"])

2025-02-23 10:15:26,181 INFO sqlalchemy.engine.Engine SELECT climate_1.year AS year, avg(climate_1.avg_temp) AS avg_temp, avg(climate_1.co2_emissions) AS co2_emissions, avg(climate_1.sea_level_rise) AS sea_level_rise, avg(climate_1.rainfall) AS avg_rainfall, avg(climate_1.renewable_energy) AS renewable_energy, avg(climate_1.extreme_weather_events) AS extreme_weather_events, avg(bls_1.value) AS avg_unemployment_rate 
FROM climate AS climate_1 JOIN bls AS bls_1 ON climate_1.year = bls_1.year GROUP BY climate_1.year ORDER BY climate_1.year
2025-02-23 10:15:26,182 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ()
